In [63]:
import pandas as pd
import numpy as np

In [64]:
train_df = pd.read_csv("../../data/rossmann-store-sales/train.csv")
test_df = pd.read_csv("../../data/rossmann-store-sales/test.csv")

/Users/bachfischer/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [65]:
train_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [66]:
def process_data(df):
    df['StateHoliday'] = df['StateHoliday'].astype(str)
    df['Open'] = df['Open'].astype(bool)
    df['Date'] = pd.to_datetime(df['Date'])

    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    df = df.rename(columns= lambda x: x.lower())
    df = df.rename(
                    columns={
                        "dayofweek": "day_of_week",
                        "stateholiday": "state_holiday",
                        "schoolholiday": "school_holiday",
                    })

    # read store data state
    df_store = pd.read_csv("../../data/rossmann-store-sales/store.csv")

    # rename columns
    df_store = df_store.rename(
                                columns={
                                    "Store": "store",
                                    "StoreType": "store_type",
                                    "Assortment": "assortment",
                                    "CompetitionDistance": "competition_distance",
                                    "CompetitionOpenSinceMonth": "competition_open_since_month",
                                    "CompetitionOpenSinceYear": "competition_open_since_year",
                                    "Promo2": "promo2",
                                    "Promo2SinceWeek": "promo2_since_week",
                                    "Promo2SinceYear": "promo2_since_year",
                                    "PromoInterval": "promo_interval",
                                })

    index = df.index
    df = pd.merge(df, df_store,how="left", on='store', indicator="_merge")
    df.index = index

    # Check that all data has been merged
    df['_merge'].eq("both").all()
    df = df.drop(columns=['_merge'])
    return df

In [67]:
train_df = process_data(train_df)
test_df = process_data(test_df)

In [68]:
train_df.head()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,year,month,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,1,5,2015-07-31,5263,555,True,1,0,1,2015,7,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,True,1,0,1,2015,7,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,True,1,0,1,2015,7,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,True,1,0,1,2015,7,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,True,1,0,1,2015,7,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [69]:
train_df['date'].max()


Timestamp('2015-07-31 00:00:00')

In [70]:
validation_window = (pd.to_datetime('2015-07-31'), pd.to_datetime('2015-03-01'))

In [71]:
select_train = (train_df['sales'].gt(0) & train_df['date'].le(validation_window[1]))
select_val = (train_df['sales'].gt(0) & train_df['date'].le(validation_window[0]) & train_df['date'].gt(validation_window[1]))

In [72]:
from sklearn.preprocessing import OrdinalEncoder

baseline_features = ["store", "day_of_week", "date", "year", "month"]

o_enc = OrdinalEncoder()
for feature in baseline_features:
    train_df[feature] = o_enc.fit_transform(train_df[[feature]])

In [73]:
# Create baseline model

X_train = train_df[select_train][baseline_features]
y_train = train_df[select_train]['sales']

In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_scaled = sc.fit_transform(X_train, y_train)
l_model = LinearRegression()
l_model.fit(X_train, y_train)

LinearRegression()

In [75]:
X_test = train_df[select_val][baseline_features]
y_test = train_df[select_val]['sales']
y_test_hat = l_model.predict(X_test)

In [78]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_test_hat, squared=False)

3048.5742260155366

array([ 8.59013665e-02, -3.10291262e+02, -2.02299311e+01,  7.60908017e+03,
        6.97735792e+02])